## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [ ]:
df_lookup = spark.read.json("/user/sprenner/lookup_table_1_official_mai.json")
df_lookup.createOrReplaceTempView("lookup")

In [7]:
spark.sql("SELECT * FROM lookup WHERE distinct_datasets > 1").toPandas()

ADLER32       BYTES                                CHILD_NAME  \
0      a4817338  389055      DAOD_JETM5.13864123._000251.pool.root.1    
1      a4817338  389055      DAOD_JETM5.13864123._000251.pool.root.1    
2      a4817338  389055      DAOD_JETM5.13864123._000251.pool.root.1    
3      a4817338  389055      DAOD_JETM5.13864123._000251.pool.root.1    
4      dd0a0e4c  521313      DAOD_JETM5.13864123._000261.pool.root.1    
5      dd0a0e4c  521313      DAOD_JETM5.13864123._000261.pool.root.1    
6      dd0a0e4c  521313      DAOD_JETM5.13864123._000261.pool.root.1    
7      dd0a0e4c  521313      DAOD_JETM5.13864123._000261.pool.root.1    
8      e4b19f66  1163369     DAOD_JETM5.13864140._000087.pool.root.1    
9      e4b19f66  1163369     DAOD_JETM5.13864140._000087.pool.root.1    
10     952a19a6  545613      DAOD_JETM5.13864169._000092.pool.root.1    
11     952a19a6  545613      DAOD_JETM5.13864169._000092.pool.root.1    
12     dc4f7c1a  707608      DAOD_JETM5.13864208._000098.pool.root.1    
13     dc4f7c1a  707608      DAOD_JETM5.13864208._000098.pool.root.1    
14     e9a3d731  1014286     DAOD_JETM5.13864217._000033.pool.root.1    
15     e9a3d731  1014286     DAOD_JETM5.13864217._000033.pool.root.1    
16     fa8c5663  837755      DAOD_JETM5.13864232._000048.pool.root.1    
17     fa8c5663  837755      DAOD_JETM5.13864232._000048.pool.root.1    
18     5d8e8bae  408122      DAOD_JETM5.13864250._000174.pool.root.1    
19     5d8e8bae  408122      DAOD_JETM5.13864250._000174.pool.root.1    
20     5d8e8bae  408122      DAOD_JETM5.13864250._000174.pool.root.1    
21     5d8e8bae  408122      DAOD_JETM5.13864250._000174.pool.root.1    
22     a18fbb54  1343100     DAOD_JETM5.13864262._000054.pool.root.1    
23     a18fbb54  1343100     DAOD_JETM5.13864262._000054.pool.root.1    
24     484817e8  917501      DAOD_JETM5.13864323._000060.pool.root.1    
25     484817e8  917501      DAOD_JETM5.13864323._000060.pool.root.1    
26     a2bb59f5  1683894     DAOD_JETM5.13864369._000088.pool.root.1    
27     a2bb59f5  1683894     DAOD_JETM5.13864369._000088.pool.root.1    
28     0f7c15ff  1161444     DAOD_JETM5.13864451._000158.pool.root.1    
29     0f7c15ff  1161444     DAOD_JETM5.13864451._000158.pool.root.1    
...         ...      ...                                         ...    
19960  16895971  1235356     DAOD_JETM5.13864556._000021.pool.root.1    
19961  16895971  1235356     DAOD_JETM5.13864556._000021.pool.root.1    
19962  16895971  1235356     DAOD_JETM5.13864556._000021.pool.root.1    
19963  16895971  1235356     DAOD_JETM5.13864556._000021.pool.root.1    
19964  cf03b081  451844      DAOD_JETM5.13864585._000082.pool.root.1    
19965  cf03b081  451844      DAOD_JETM5.13864585._000082.pool.root.1    
19966  34842999  452329      DAOD_JETM5.13864592._000114.pool.root.1    
19967  34842999  452329      DAOD_JETM5.13864592._000114.pool.root.1    
19968  34842999  452329      DAOD_JETM5.13864592._000114.pool.root.1    
19969  34842999  452329      DAOD_JETM5.13864592._000114.pool.root.1    
19970  2f041373  278157727   DAOD_EXOT17.14017369._000010.pool.root.1   
19971  2f041373  278157727   DAOD_EXOT17.14017369._000010.pool.root.1   
19972  3521a060  188395828   DAOD_EXOT17.14024361._000006.pool.root.1   
19973  3521a060  188395828   DAOD_EXOT17.14024361._000006.pool.root.1   
19974  469109c1  443102484   DAOD_EXOT17.14118483._000100.pool.root.1   
19975  469109c1  443102484   DAOD_EXOT17.14118483._000100.pool.root.1   
19976  0412b473  2418189651  DAOD_SUSY6.14024456._000060.pool.root.1    
19977  0412b473  2418189651  DAOD_SUSY6.14024456._000060.pool.root.1    
19978  0412b473  2418189651  DAOD_SUSY6.14024456._000060.pool.root.1    
19979  0412b473  2418189651  DAOD_SUSY6.14024456._000060.pool.root.1    
19980  7728efa5  551808635   DAOD_STDM5.14057993._000010.pool.root.1    
19981  7728efa5  551808635   DAOD_STDM5.14057993._000010.pool.root.1    
19982  09983cf1  4446819992  DAOD_SUSY1.14120661._000015.pool.root

In [8]:
df_mai = spark.read.json("/user/sprenner/training_official_mai.json")
df_mai = df_mai.withColumn("join_key_mai", concat_ws(':', df_mai["scope"], df_mai["filename"]))
df_mai = df_mai.withColumn("fileScope", df_mai["scope"])
df_mai = df_mai.drop("scope")
df_mai.createOrReplaceTempView("mai")

In [9]:
df_lookup = df_lookup.dropDuplicates(['BYTES','CHILD_NAME','CHILD_SCOPE','CHILD_TYPE','DID_TYPE','NAME','SCOPE'])
df_lookup.createOrReplaceTempView("lookup")
df_lookup = spark.sql("SELECT * FROM lookup WHERE SCOPE != 'hc_test' AND distinct_datasets == 1") # zum checken distinct_datasets rausnehmen
df_lookup.createOrReplaceTempView("lookup")
multiple = spark.sql("SELECT join_key_mai AS join_key_mai_multi, COUNT(*) as count\
           FROM lookup \
           GROUP BY join_key_mai\
           HAVING count > 1 \
           ORDER BY 2 DESC")
multiple.toPandas()

Empty DataFrame
Columns: [join_key_mai_multi, count]
Index: []

In [10]:
df_lookup.describe().toPandas()

summary   ADLER32                BYTES                          CHILD_NAME  \
0  count   333533    333533               333533                               
1  mean    Infinity  8.435781420026114E8  None                                 
2  stddev  NaN       1.437717146123461E9  None                                 
3  min     0000180a  10000                AOD.05341490._000446.pool.root.1     
4  max     fff0e38f  99999162             log.14217468._003414.job.log.tgz.1   

   CHILD_SCOPE CHILD_TYPE             CREATED_AT DID_TYPE             EVENTS  \
0  333533       333533     333533                 333533   293946              
1  None         None       1.5095846112173032E12  None     46496.86479829629   
2  None         None       3.135251343712882E10   None     785874.3769434602   
3  data12_8TeV  F          1331912599000          D        0                   
4  mc16_valid   F          1527733358000          D        9999500             

  LENGTH  \
0  8233    
1  0.0     
2  0.0     
3  0       
4  0       

                                                                                                             NAME  \
0  333533                                                                                                           
1  None                                                                                                             
2  None                                                                                                             
3  data12_8TeV.00200805.physics_MinBias.recon.NTUP_MINBIAS.r4768_tid01304347_00                                     
4  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_minbias_inelastic_low.merge.HITS.e3581_s3028_s2939_tid10070623_00   

  RULE_EVALUATION        SCOPE             UPDATED_AT   account  \
0  8233            333533       333533                 333533     
1  0.0             None         1.5095848615165781E12  None       
2  0.0             None         3.1352603826837093E10  None       
3  0               data12_8TeV  1331912599000          abell      
4  0               mc16_valid   1527733386000          zmarshal   

             accountHash         clientState       clientStateHash  \
0  333533                 333533              333533                 
1  -1.3201178022393286E8  None                -9.762384589943874E8   
2  1.0416362153054254E9   None                2.1222553731322375E8   
3  -2123889579            CannotAuthenticate  -1913538301            
4  2140382084             SourceNotFound      634051254              

                                                                                           dataset  \
0  333533                                                                                            
1  None                                                                                              
2  None                                                                                              
3  data12_8TeV.00200805.physics_MinBias.recon.NTUP_MINBIAS.r4768                                     
4  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_minbias_inelastic_low.merge.HITS.e3581_s3028_s2939   

             datasetHash datasetScope     datasetScopeHash dataset_version  \
0  333533                 333533       333533               333533           
1  -2.3190460577007372E7  None         -8.02612486784678E8  None             
2  1.2451711952222888E9   None         7.729675153671582E8  None             
3  -2147401420            data12_8TeV  -2057006107          c1128_m1842      
4  2147445885             mc16_valid   2109319881           x563_m1972       

    dataset_versionHash datatype          datatypeHash distinct_datasets  \
0  333533                333533   333533                333533             
1  2.002699956153364E7   None     1.2947054584258829E8  1.0                
2  1.1460548810232995E9  None     1.0631003035015748E9  0.0                
3  -2144917680           AOD      -2070622869         

In [11]:
#df_lookup = df_lookup.withColumn('dataset_lookup',when(col("distinct_datasets").isNull(), df_lookup["dataset"]).otherwise(df_lookup["NAME"]))

In [12]:
df_datasets_lookup = df_lookup.select("NAME","SCOPE","DID_TYPE","CHILD_NAME","CHILD_SCOPE", "join_key_lookup")

In [14]:
df_datasets_lookup.write.format("json").save("/user/sprenner/lookup_table_2_official_mai.json")